Prompt Chaining + Iterative Generation 
- break large problems into smaller, interconnected prompts 
- the output becomes the input for the next chain 

In [15]:
from dotenv import load_dotenv
import google.generativeai as genai
import os
from google.api_core import retry
from pprint import pprint

In [3]:
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")

In [4]:
genai.configure(api_key=google_api_key)

In [6]:
from utils import SAFETY_SETTINGS, MODEL

In [7]:
model = genai.GenerativeModel(MODEL)
chat = model.start_chat() 

In [9]:
chat.send_message("Tell me the difference between AI agents and multi chaining. Format it nicely in a bullet point list")

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=glm.GenerateContentResponse({'candidates': [{'content': {'parts': [{'text': '## AI Agents vs. Multi-Chaining: Key Differences\n\n* **Function:**\n    * **AI Agents:** Perform actions and make decisions within an environment to achieve goals.\n    * **Multi-Chaining:** Processes information and generates text through a sequence of interconnected prompts. \n* **Scope:**\n    * **AI Agents:** Independent entities capable of autonomous operation.\n    * **Multi-Chaining:** A technique used within large language models (LLMs) for prompt engineering.\n* **Purpose:**\n    * **AI Agents:** Achieve specific objectives, such as navigating a car, providing customer service, or managing finances. \n    * **Multi-Chaining:** Improve the accuracy and efficiency of LLMs in handling complex tasks and generating different creative text formats. \n* **Complexity:**\n    * **AI Agents:** Can exhibit complex behaviors, learnin

# AI Agents vs. Multi-Chaining: Key Differences

## Function
- **AI Agents:** Perform actions and make decisions within an environment to achieve goals.
- **Multi-Chaining:** Processes information and generates text through a sequence of interconnected prompts.

## Scope
- **AI Agents:** Independent entities capable of autonomous operation.
- **Multi-Chaining:** A technique used within large language models (LLMs) for prompt engineering.

## Purpose
- **AI Agents:** Achieve specific objectives, such as navigating a car, providing customer service, or managing finances.
- **Multi-Chaining:** Improve the accuracy and efficiency of LLMs in handling complex tasks and generating different creative text formats.

## Complexity
- **AI Agents:** Can exhibit complex behaviors, learning and adapting to their environment.
- **Multi-Chaining:** Follows a structured sequence of steps, with each prompt building upon the previous one.

## Examples
- **AI Agents:** Chatbots, self-driving cars, recommendation systems, trading bots.
- **Multi-Chaining:** Summarizing long articles, writing poems or scripts, translating languages with specific styles.

In short, AI agents are active doers, while multi-chaining is a method for guiding LLMs to think step-by-step.


In [19]:
def generate_with_retry(prompt):
  return model.generate_content(prompt, request_options={'retry':retry.Retry()})

In [17]:
persona = '''\
You are a leading AI researcher with extensive experience in designing and developing intelligent agents. 
Your goal is to write a comprehensive and insightful article on the current state and future potential of AI agents.'''

In [26]:
article_text = generate_with_retry(persona).text
print(article_text)

## The Rise of Intelligent Agents: Navigating the Present and Charting the Future

The field of Artificial Intelligence (AI) has witnessed an explosive growth in recent years, with intelligent agents emerging as a pivotal area of focus. These autonomous entities, capable of perceiving their environment, learning from experience, and taking actions to achieve goals, hold immense promise for revolutionizing numerous aspects of our lives. This article delves into the current state of AI agents, explores their potential applications, and contemplates the challenges and opportunities that lie ahead.

### The State of the Art: A Diverse Landscape of Intelligent Agents

Currently, AI agents exhibit a wide spectrum of capabilities depending on their design and purpose. We see them powering various applications, including:

* **Virtual Assistants:** From scheduling appointments to controlling smart home devices, AI assistants like Siri and Google Assistant leverage natural language processing a

In [20]:
premise_prompt = f'''\
{persona}

Identify a key area within the field of AI agents to explore in this article. This could be a specific application, 
a technical challenge, or a societal impact of AI agents.'''

In [21]:
outline_prompt = f'''\
{persona}

The article will focus on:

{{premise}}

Develop a structured outline for the article, including key sections and subtopics to address.'''

In [22]:
continuation_prompt = f'''\
{persona}

This article explores:

{{premise}}

Following this outline:

{{outline}}

Here's the content written so far:

{{article_text}}

=====

Review the outline and the existing content. Determine the next section or subtopic to address. 

Expand on the article by writing the next section in detail. Aim for at least 500 words, providing in-depth analysis, 
examples, and evidence to support your points. Write ENDOFSECTION when the section is complete.

Remember, the article should be comprehensive, insightful, and engaging for readers interested in AI agents.'''

In [23]:
# now I will generate the premise 
premise = generate_with_retry(premise_prompt).text
print(premise) 

## Exploring the Societal Impact of AI Agents: A Focus on the Future of Work

The field of AI agents is rapidly evolving, with advancements in machine learning and other related technologies leading to increasingly sophisticated and capable agents. As these agents become more integrated into our lives, it's crucial to examine their potential societal impact, particularly on the future of work. This article will delve into this critical area, exploring both the challenges and opportunities presented by the rise of AI agents in the workforce.

**Impact on Job Automation:**

*   **Job displacement:** One of the primary concerns surrounding AI agents is their potential to automate tasks currently performed by humans, leading to job displacement across various industries. This displacement could disproportionately affect certain sectors, such as manufacturing, transportation, and customer service. 
*   **Job creation:** Conversely, the development and implementation of AI agents also create

In [24]:
# now I will generate the outline 
outline = generate_with_retry(outline_prompt.format(premise=premise)).text 
print(outline)

## Article Outline: Exploring the Societal Impact of AI Agents: A Focus on the Future of Work

**I. Introduction**

*   The Rise of AI Agents: Briefly introduce the advancements in AI and the increasing sophistication of AI agents.
*   Focus on the Future of Work: Highlight the significance of examining the impact of AI agents on the workforce and its implications for society.
*   Article Overview: Briefly outline the key topics and structure of the article.

**II. Impact on Job Automation**

*   Job displacement: 
    *   Discuss the potential for AI agents to automate tasks and displace human workers across various industries.
    *   Provide examples of sectors particularly vulnerable to automation (e.g., manufacturing, transportation, customer service).
*   Job creation:
    *   Explore the creation of new job opportunities related to AI development, data science, and robot maintenance.
    *   Discuss how automation of routine tasks can enable humans to focus on higher-level, crea

In [28]:
# initialise next_section as an empty string 
next_section = ""

In [29]:
while "ENDOFSECTION" in next_section:
    next_section = generate_with_retry(continuation_prompt.format(premise=premise, outline=outline, article_text=article_text)).text
    article_text = article_text + "\n\n" + next_section

In [30]:
final_article = article_text.replace("ENDOFSECTION", "").strip()
print(final_article)

## The Rise of Intelligent Agents: Navigating the Present and Charting the Future

The field of Artificial Intelligence (AI) has witnessed an explosive growth in recent years, with intelligent agents emerging as a pivotal area of focus. These autonomous entities, capable of perceiving their environment, learning from experience, and taking actions to achieve goals, hold immense promise for revolutionizing numerous aspects of our lives. This article delves into the current state of AI agents, explores their potential applications, and contemplates the challenges and opportunities that lie ahead.

### The State of the Art: A Diverse Landscape of Intelligent Agents

Currently, AI agents exhibit a wide spectrum of capabilities depending on their design and purpose. We see them powering various applications, including:

* **Virtual Assistants:** From scheduling appointments to controlling smart home devices, AI assistants like Siri and Google Assistant leverage natural language processing a